In [435]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import json
from copy import copy

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

Autosaving every 300 seconds


# Data Cleaning and Preliminary Exploration

## Preliminary Exploration

In [436]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36926,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.498289,5.96721,6,6,2017-08-13 23:50:48.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.999316,11.96720,12,12,2017-08-13 23:50:48.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,1.998630,23.93440,24,24,2017-08-13 23:50:48.0


#### NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
#### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

In [437]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)
ADNI2.head()


(6937, 94)


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,...,1.00000,1.25641,1.13549,NaN,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,...,1.33333,1.37838,1.50629,NaN,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


In [438]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)

In [439]:
# calculate the % of each column that is NaNs and print in descending order
nan_series = (ADNI2.isnull().sum()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(nan_series)

PIB                         1.000000
PIB_bl                      0.992936
FDG                         0.806833
AV45                        0.726251
FLDSTRENG                   0.553986
MidTemp                     0.543895
Fusiform                    0.543895
Entorhinal                  0.543895
Ventricles                  0.500793
Hippocampus                 0.497189
WholeBrain                  0.472106
FSVERSION                   0.441401
ICV                         0.441401
EcogSPOrgan                 0.363269
EcogPtOrgan                 0.356927
EcogSPDivatt                0.352314
MOCA                        0.350584
RAVLT_perc_forgetting       0.349575
EcogPtVisspat               0.347268
EcogPtDivatt                0.346836
EcogSPVisspat               0.346259
RAVLT_forgetting            0.345827
RAVLT_learning              0.344385
RAVLT_immediate             0.344385
EcogPtPlan                  0.342944
EcogSPPlan                  0.342655
EcogPtLang                  0.341790
A

In [440]:
# calculate the % of each column that is -1 and print in descending order
minus_one_series = (ADNI2[ADNI2 == -1].count()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(minus_one_series)

RAVLT_forgetting_bl         0.012830
RAVLT_learning              0.009658
RAVLT_learning_bl           0.008217
RAVLT_forgetting            0.007496
EcogSPTotal                 0.000000
EcogSPDivatt                0.000000
RAVLT_perc_forgetting       0.000000
FAQ                         0.000000
MOCA                        0.000000
EcogPtMem                   0.000000
EcogPtLang                  0.000000
EcogPtVisspat               0.000000
EcogPtPlan                  0.000000
EcogPtOrgan                 0.000000
FSVERSION                   0.000000
FLDSTRENG                   0.000000
Hippocampus                 0.000000
EcogPtTotal                 0.000000
EcogSPMem                   0.000000
EcogSPLang                  0.000000
EcogSPVisspat               0.000000
EcogSPPlan                  0.000000
EcogSPOrgan                 0.000000
EcogPtDivatt                0.000000
update_stamp                0.000000
RAVLT_immediate             0.000000
PTEDUCAT                    0.000000
P

In [441]:
# calculate the % of each column that is -4 and print in descending order
minus_one_series = (ADNI2[ADNI2 == -4].count()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(minus_one_series)

RAVLT_forgetting            0.000288
RAVLT_learning              0.000288
RAVLT_forgetting_bl         0.000144
EcogPtDivatt                0.000000
RAVLT_perc_forgetting       0.000000
FAQ                         0.000000
MOCA                        0.000000
EcogPtMem                   0.000000
EcogPtLang                  0.000000
EcogPtVisspat               0.000000
EcogPtPlan                  0.000000
EcogPtOrgan                 0.000000
update_stamp                0.000000
EcogSPMem                   0.000000
EcogSPLang                  0.000000
EcogSPVisspat               0.000000
EcogSPPlan                  0.000000
EcogSPOrgan                 0.000000
EcogSPDivatt                0.000000
EcogSPTotal                 0.000000
FLDSTRENG                   0.000000
FSVERSION                   0.000000
EcogPtTotal                 0.000000
RAVLT_immediate             0.000000
Hippocampus                 0.000000
PTEDUCAT                    0.000000
PTID                        0.000000
V

#### Notes
- Our pure dataframe is called ADNI2

## Removing All Missing Data

In [442]:
ADNI2_remove_all_missing = copy(ADNI2)
for column in ADNI2_remove_all_missing.columns:
    if ADNI2_remove_all_missing[column].isnull().any():
        ADNI2_remove_all_missing = ADNI2_remove_all_missing.drop(labels=column, axis=1)

In [443]:
ADNI2_remove_all_missing.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,EXAMDATE_bl,CDRSB_bl,MMSE_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,17,2013-09-09,0.0,29,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,16,2013-09-17,0.0,30,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,16,2013-09-17,0.0,30,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,16,2013-09-17,0.0,30,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,20,2013-10-23,0.0,30,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


In [444]:
ADNI2_remove_all_missing.shape

(6937, 19)

In [445]:
ADNI2_remove_all_missing.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'EXAMDATE_bl', 'CDRSB_bl',
       'MMSE_bl', 'Years_bl', 'Month_bl', 'Month', 'M', 'update_stamp'],
      dtype='object')

In [446]:
categorical_predictors = ['VISCODE', 'PTGENDER', 'PTEDUCAT']
continuous_predictors = ['RID', 'PTID', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE', 'DX_bl', 'AGE', 
                         'EXAMDATE_bl', 'CDRSB_bl', 'MMSE_bl', 'Years_bl', 'Month_bl', 'Month', 
                         'M', 'update_stamp']
#dataframe_list = []
categorical_columns_g2cols = ['PTGENDER']                                 
ADNI2_remove_all_missing_clean  = pd.get_dummies(ADNI2_remove_all_missing, columns=categorical_columns_g2cols, prefix = categorical_columns_g2cols , drop_first=False)
# for i in range(len(categorical_predictors)):
#     dummy_frame = pd.get_dummies(ADNI2[categorical_predictors[i]])
#     # The commemted line below could be adjust, I just want to make the columns name clearer
#     #dummy_frame = dummy_frame.iloc[:, :-1]
#     dataframe_list.append(dummy_frame)
# ADNI2_categorical = pd.concat(dataframe_list, axis=1)
# ADNI2_remove_all_missing_clean = pd.concat([ADNI2_remove_all_missing[continuous_predictors], \
                                            #ADNI2_categorical], axis=1)

In [447]:
print(ADNI2_remove_all_missing_clean.shape)
ADNI2_remove_all_missing_clean.head()


(6937, 20)


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTEDUCAT,EXAMDATE_bl,CDRSB_bl,MMSE_bl,Years_bl,Month_bl,Month,M,update_stamp,PTGENDER_Female,PTGENDER_Male
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,17,2013-09-09,0.0,29,0.000000,0.00000,0,0,2017-10-06 23:20:00.0,0,1
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,16,2013-09-17,0.0,30,2.031490,24.32790,24,24,2017-10-06 23:20:00.0,0,1
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,16,2013-09-17,0.0,30,0.465435,5.57377,6,6,2017-10-06 23:20:00.0,0,1
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,16,2013-09-17,0.0,30,0.000000,0.00000,0,0,2017-10-06 23:20:00.0,0,1
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,20,2013-10-23,0.0,30,0.000000,0.00000,0,0,2017-10-06 23:20:00.0,0,1


In [448]:
ADNI2_remove_all_missing_clean.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTEDUCAT', 'EXAMDATE_bl', 'CDRSB_bl', 'MMSE_bl',
       'Years_bl', 'Month_bl', 'Month', 'M', 'update_stamp', 'PTGENDER_Female',
       'PTGENDER_Male'],
      dtype='object')

In [449]:
ADNI2_remove_all_missing_clean.to_csv(path_or_buf='ADNI2_remove_all_missing_v3.csv', index=False)

## Model-Based Imputation 
- Drop around 5% of Categorical predictors and use Linear Regression for missing continuous predictors

In [450]:
for column in ADNI2.columns:
    print(column, ADNI2[column].unique())

RID [5282 5280 5279 ..., 5294 5295 5296]
PTID ['082_S_5282' '100_S_5280' '082_S_5279' ..., '051_S_5294' '057_S_5295'
 '053_S_5296']
VISCODE ['bl' 'm24' 'm06' 'm03' 'm36' 'm12' 'm48' 'm18' 'm30' 'm42' 'm54' 'm60'
 'm72' 'm66' 'm108' 'm96' 'm84' 'm78' 'm90' 'm120' 'm114' 'm102' 'm126']
SITE [ 82 100  27 135  53  36 153 127   7  32  18  33 130   2  41   9  16 126
  19  23  21 114  73  37  29  12  67   3  72  68  20  57 941  13   6 131
 128   5  52  24  70  51  22  99 141  11  31 123 116 137  94  35  14 109
 136  98  10 129]
COLPROT ['ADNI2']
ORIGPROT ['ADNI2' 'ADNIGO' 'ADNI1']
EXAMDATE ['2013-09-09' '2015-09-29' '2014-03-06' ..., '2017-08-16' '2016-08-22'
 '2015-09-30']
DX_bl ['SMC' 'AD' 'EMCI' 'LMCI' 'CN']
AGE [ 66.9  67.5  68.5  80.2  71.8  78.   90.1  69.9  80.4  64.9  65.8  65.9
  75.1  75.2  73.5  78.1  78.8  77.9  70.4  68.1  56.5  66.1  73.9  74.3
  75.8  71.   67.2  87.8  62.7  80.3  85.3  68.   78.6  74.2  77.8  71.3
  75.7  82.5  69.6  65.4  78.2  85.8  76.1  68.7  74.9  85.2  5

Entorhinal_bl [   nan  4946.  4065.  4313.  4043.  3375.  3772.  4315.  3146.  4766.
  4034.  3619.  4174.  3999.  3535.  2632.  4239.  5275.  3111.  3007.
  3568.  3671.  4206.  4138.  3592.  3143.  3082.  3586.  3537.  3388.
  4088.  1757.  4133.  4310.  3302.  3116.  2671.  2602.  3773.  2755.
  2523.  3066.  4527.  4587.  3292.  4220.  3304.  4163.  4162.  2928.
  4467.  3964.  3623.  3801.  3474.  5363.  2615.  4797.  3508.  3739.
  3587.  2382.  3863.  3615.  4706.  5019.  4736.  3842.  5116.  4046.
  3026.  2694.  4149.  3754.  3684.  4509.  4061.  4606.  3884.  3691.
  3714.  3303.  4036.  2588.  2103.  4353.  3299.  3812.  4203.  2725.
  4728.  3491.  3328.  3421.  3770.  3507.  3320.  2655.  2851.  3976.
  4146.  3994.  3309.  2766.  3820.  2473.  3857.  4170.  2461.  3381.
  4294.  3540.  3071.  2147.  2646.  4357.  3154.  3594.  3886.  2176.
  2910.  3229.  2518.  3289.  3900.  1883.  4436.  3599.  3025.  2170.
  2123.  2535.  3105.  3929.  3277.  2066.  2138.  3098.  4661.

In [451]:
ADNI2.columns

Index(['RID', 'PTID', 'VISCODE', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',
       'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT',
       'PTMARRY', 'APOE4', 'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'FLDSTRENG', 'FSVERSION',
       'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl

In [452]:
all_predictors = ['VISCODE','DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4',
                  'FDG', 'PIB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate', 
                  'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 
                  'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 
                  'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt', 
                  'EcogSPTotal', 'FLDSTRENG', 'FSVERSION', 'Ventricles', 'Hippocampus', 'WholeBrain', 
                  'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl', 
                  'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 
                  'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 
                  'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                  'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl', 
                  'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 
                  'EcogSPPlan_bl', 'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'PIB_bl', 
                  'AV45_bl', 'Years_bl']
continuous_predictors = ['AGE',
                  'FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate', 
                  'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 
                  'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 
                  'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt', 
                  'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 
                  'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'CDRSB_bl', 'ADAS11_bl', 
                  'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 
                  'RAVLT_perc_forgetting_bl', 'FAQ_bl', 'Ventricles_bl', 
                  'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
                  'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl', 
                  'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl', 'EcogSPVisspat_bl', 
                  'EcogSPPlan_bl', 'EcogSPOrgan_bl', 'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 
                  'AV45_bl', 'Years_bl']

In [453]:
''' Pre-Processing the Missing Categorical Data '''
categorical_predictors = ['DX_bl', 'VISCODE','PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'FLDSTRENG', 'FSVERSION',
                          'DX']



''' Remove all categorical data with nan (only ~0.5%) '''
missing_rows = np.nonzero(np.sum(ADNI2[categorical_predictors].isnull(), axis=1))
ADNI2_categorical_clean = ADNI2.drop(ADNI2.index[missing_rows[0]])

''' Hot-One Encoding of Categorical Predictors '''
# dataframe_list = []
# for i in range(len(categorical_predictors)):
#     dummy_frame = pd.get_dummies(ADNI2[categorical_predictors[i]])
#     # The commemted line below could be adjust, I just want to make the columns name clearer
#     #dummy_frame = dummy_frame.iloc[:, :-1]
#     dataframe_list.append(dummy_frame)


# ADNI2_categorical = pd.concat(dataframe_list, axis=1)
# ADNI2_categorical.rename(columns={0: "APOE4_1", 1: "APOE4_2"}, inplace=True);

ADNI2_categorical = pd.get_dummies(ADNI2[categorical_predictors] , columns=categorical_predictors, prefix = categorical_predictors , drop_first=False)
    

In [454]:
ADNI2_categorical.head()

,DX_bl_AD,DX_bl_CN,DX_bl_EMCI,DX_bl_LMCI,DX_bl_SMC,VISCODE_bl,VISCODE_m03,VISCODE_m06,VISCODE_m102,VISCODE_m108,...,APOE4_0.0,APOE4_1.0,APOE4_2.0,FLDSTRENG_1.5 Tesla MRI,FLDSTRENG_3 Tesla MRI,FSVERSION_Cross-Sectional FreeSurfer (5.1),FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),DX_CN,DX_Dementia,DX_MCI
58,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,1,0,1,0,0
59,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
68,0,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
91,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,1,1,0,1,0,0
92,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,1,1,0,1,0,0


In [455]:
ADNI2_categorical.columns

Index(['DX_bl_AD', 'DX_bl_CN', 'DX_bl_EMCI', 'DX_bl_LMCI', 'DX_bl_SMC',
       'VISCODE_bl', 'VISCODE_m03', 'VISCODE_m06', 'VISCODE_m102',
       'VISCODE_m108', 'VISCODE_m114', 'VISCODE_m12', 'VISCODE_m120',
       'VISCODE_m126', 'VISCODE_m18', 'VISCODE_m24', 'VISCODE_m30',
       'VISCODE_m36', 'VISCODE_m42', 'VISCODE_m48', 'VISCODE_m54',
       'VISCODE_m60', 'VISCODE_m66', 'VISCODE_m72', 'VISCODE_m78',
       'VISCODE_m84', 'VISCODE_m90', 'VISCODE_m96', 'PTGENDER_Female',
       'PTGENDER_Male', 'PTETHCAT_Hisp/Latino', 'PTETHCAT_Not Hisp/Latino',
       'PTRACCAT_Am Indian/Alaskan', 'PTRACCAT_Asian', 'PTRACCAT_Black',
       'PTRACCAT_Hawaiian/Other PI', 'PTRACCAT_More than one',
       'PTRACCAT_White', 'PTMARRY_Divorced', 'PTMARRY_Married',
       'PTMARRY_Never married', 'PTMARRY_Widowed', 'APOE4_0.0', 'APOE4_1.0',
       'APOE4_2.0', 'FLDSTRENG_1.5 Tesla MRI', 'FLDSTRENG_3 Tesla MRI',
       'FSVERSION_Cross-Sectional FreeSurfer (5.1)',
       'FSVERSION_Cross-Sectional FreeSu

In [456]:
''' Create ADNI2_missing for imputation '''
#ADNI2_remove_categorical = ADNI2[[column for column in ADNI2.columns if column not in categorical_predictors]]
ADNI2_remove_categorical = ADNI2[continuous_predictors]

''' Create the Dataframe for imputation '''
ADNI2_missing = pd.concat([ADNI2_remove_categorical, ADNI2_categorical], axis=1)

In [457]:
print(ADNI2_remove_categorical.shape)
print(ADNI2_categorical.shape)

(6937, 68)
(6937, 52)


In [458]:
ADNI2_missing.head()

,AGE,FDG,AV45,CDRSB,ADAS11,ADAS13,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,...,APOE4_0.0,APOE4_1.0,APOE4_2.0,FLDSTRENG_1.5 Tesla MRI,FLDSTRENG_3 Tesla MRI,FSVERSION_Cross-Sectional FreeSurfer (5.1),FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3),DX_CN,DX_Dementia,DX_MCI
58,66.9,1.13549,1.326790,0.0,10.0,15.0,29.0,42.0,5.0,2.0,...,0,1,0,0,1,1,0,1,0,0
59,67.5,NaN,0.982566,0.0,2.0,3.0,30.0,38.0,6.0,5.0,...,1,0,0,0,0,0,0,1,0,0
68,67.5,NaN,NaN,0.0,3.0,4.0,29.0,54.0,6.0,2.0,...,1,0,0,0,0,0,0,1,0,0
91,67.5,1.25195,0.983143,0.0,3.0,5.0,30.0,42.0,7.0,2.0,...,1,0,0,0,1,1,0,1,0,0
92,68.5,1.50629,0.985156,0.0,4.0,7.0,30.0,61.0,6.0,3.0,...,1,0,0,0,1,1,0,1,0,0


In [459]:
print(ADNI2_missing.shape)

(6937, 120)


In [460]:
full_dataframe =  copy(ADNI2_missing)
missing_rows = []
for i in range(ADNI2_missing.shape[0]):
    if pd.isnull(ADNI2_missing.values[i]).any():
        missing_rows.append(i)
full_dataframe = full_dataframe.drop(full_dataframe.index[missing_rows], axis=0);

print('The origin number of data: {}'.format(len(ADNI2_missing)))
print('The number of data that contains missing entries: {}'.format(len(missing_rows)))
print('The dimension of full dataframe: {}'.format(full_dataframe.shape))

The origin number of data: 6937
The number of data that contains missing entries: 6263
The dimension of full dataframe: (674, 120)


In [461]:
''' To check that full_dataframe has no missing values '''
np.sum(np.sum(full_dataframe.isnull()))

0

In [462]:
np.sum(np.sum(full_dataframe.isnull()))

0

In [463]:
''' Model-Based Imputation - Linear Regression '''

from sklearn.metrics import mean_squared_error

lr = LinearRegression(fit_intercept = True)

for i in range(ADNI2_missing.shape[0]):
    if i % 1000 == 0:
        print(i)
    if ADNI2_missing.iloc[i, :].isnull().any():
        missing_columns = ADNI2_missing.columns[ADNI2_missing.iloc[i, :].isnull()]
        print(i, missing_columns)
        y_imp = full_dataframe[missing_columns]
        X_imp = full_dataframe.drop(missing_columns, 1)
        lr.fit(X_imp,y_imp)
        X_missing = ADNI2_missing.iloc[i, :].drop(missing_columns)
        y_hat = lr.predict(X_missing)
        ADNI2_missing.loc[ADNI2_missing.index[i], missing_columns] = y_hat[0] 
     

0
0 Index(['MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'Ventricles',
       'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'Ventricles_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', 'EcogPtDivatt_bl', 'EcogPtTotal_bl'],
      dtype='object')
1 Index(['FDG', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp',
       'ICV'],
      dtype='object')
2 Index(['FDG', 'AV45', 'EcogSPOrgan', 'Ventricles', 'Hippocampus', 'WholeBrain',
       'Entorhinal', 'Fusiform', 'MidTemp', 'ICV'],
      dtype='object')
5 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dtype=

66 Index(['FDG'], dtype='object')
67 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dtype='object')
69 Index(['FDG', 'Hippocampus_bl'], dtype='object')
70 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'Hippocampus_bl'],
      dtype='object')
71 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'Hippocampus_bl'],
      dtype='object')
72 Index(['Hippocampus', 'Hippocampus_bl'], dtype='object')
73 Index(['FDG'

131 Index(['Ventricles', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp',
       'Ventricles_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl'],
      dtype='object')
132 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'Hippocampus_bl'],
      dtype='object')
133 Index(['Hippocampus', 'Hippocampus_bl'], dtype='object')
134 Index(['FDG', 'AV45', 'EcogSPPlan_bl'], dtype='object')
135 Index(['FDG', 'AV45', 'EcogSPPlan_bl'], dtype='object')
136 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'EcogSPPlan_bl'],
      dtype='object')
137 Index(['EcogSPPlan

172 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles_bl', 'WholeBrain_bl', 'Entorhinal_bl',
       'Fusiform_bl', 'MidTemp_bl', 'MOCA_bl'],
      dtype='object')
173 Index(['MOCA', 'Ventricles', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp',
       'Ventricles_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'MOCA_bl'],
      dtype='object')
174 Index(['FDG', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dtype='object')
175 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dt

220 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp'],
      dtype='object')
222 Index(['FDG'], dtype='object')
223 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV'],
      dtype='object')
224 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'Ecog

...

6793 Index(['Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform',
       'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'AV45_bl'],
      dtype='object')
6794 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl'],
      dtype='object')
6795 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'IC

6821 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl'],
      dtype='object')
6822 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'M

6847 Index(['FDG', 'AV45', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'MOCA',
       'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan',
       'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'AV45_bl'],
      dtype='object')
6848 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPt

6872 Index(['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE', 'RAVLT_immediate',
       'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ',
       'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan',
       'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'FDG_bl', 'AV45_bl'],
      dtype='object')
6873 Index(['FDG', 'AV45', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTot

6897 Index(['FDG', 'AV45', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_bl',
       'EcogSPVisspat_bl', 'EcogSPPlan_bl', 'EcogSPOrgan_bl',
       'EcogSPDivatt_bl', 'EcogSPTotal_bl', 'AV45_bl'],
      dtype='object')
6898 Index(['FDG', 'AV45', 'ADAS11', 'ADAS13', 'RAVLT_immediate', 'RAVLT_learning',
       'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'MOCA', 'EcogPtMem',
       'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan',
       'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal',
       'Fusiform', 'MidTemp', 'ICV', 'MOCA_bl', 'EcogPtMem_bl',
       'EcogPtLang_bl', 'EcogPtVisspat_bl', 'EcogPtPlan

In [464]:
ADNI2_missing.shape

(6937, 120)

In [465]:
ADNI2_missing.columns[0:3]

Index(['AGE', 'FDG', 'AV45'], dtype='object')

In [466]:
np.sum(np.sum(ADNI2_missing.isnull()))

0

In [467]:
ADNI2_missing.to_csv(path_or_buf='ADNI2_mb_impute_v3.csv', index=False)

In [468]:
#ADNI2_missing.to_csv(path_or_buf='ADNI2_mb_impute.csv', index=False)

# Below are More Preliminary Exploration by Carlo

In [9]:
## Drop the predictors that have more than 50% missing (thresh parameter)

ADNI2_clean= ADNI2.dropna(axis=1, how='any', thresh=len(ADNI2['DX_bl'])/2, inplace=False)
print("Shape of ADNI2 before dropping columns with NaN>50%")
print(ADNI2.shape)
print("\n")
print("Shape of ADNI2 after dropping columns with NaN>50%")
print(ADNI2_clean.shape)

Shape of ADNI2 before dropping columns with NaN>50%
(6937, 94)


Shape of ADNI2 after dropping columns with NaN>50%
(6937, 85)


In [46]:
ADNI2_clean_remove_all_missing = ADNI2.dropna(axis=1, how='any', thresh=len(ADNI2['DX_bl']), inplace=False)

In [47]:
ADNI2_clean_remove_all_missing.shape

(6937, 19)

In [49]:
np.sum(np.sum(ADNI2_clean_remove_all_missing.isnull()))

0

In [32]:
ADNI2_clean_remove_all_missing.to_json(path_or_buf='ADNI2_clean_remove_all_missing.json' )

In [36]:
with open("ADNI2_clean_remove_all_missing.json") as json_file: 
    ADNI2_clean_remove_all_missing_load = pd.read_json(path_or_buf='ADNI2_clean_remove_all_missing.json')

In [38]:
ADNI2_clean_remove_all_missing_load.head()

,AGE,CDRSB_bl,COLPROT,DX_bl,EXAMDATE,EXAMDATE_bl,M,MMSE_bl,Month,Month_bl,ORIGPROT,PTEDUCAT,PTGENDER,PTID,RID,SITE,VISCODE,Years_bl,update_stamp
10000,55.0,1.0,ADNI2,LMCI,2012-01-26,2011-07-27,6,28,6,6.00000,ADNI2,14,Female,035_S_4114,4114,35,m06,0.501027,2017-10-06 23:19:54.0
10001,55.0,1.0,ADNI2,LMCI,2011-11-02,2011-07-27,3,28,3,3.21311,ADNI2,14,Female,035_S_4114,4114,35,m03,0.268309,2017-08-29 23:07:16.0
10002,55.0,1.0,ADNI2,LMCI,2011-07-27,2011-07-27,0,28,0,0.00000,ADNI2,14,Female,035_S_4114,4114,35,bl,0.000000,2017-10-06 23:19:54.0
10003,70.8,0.0,ADNI2,CN,2015-07-29,2011-07-22,48,30,48,48.13110,ADNI2,18,Female,011_S_4105,4105,11,m48,4.019170,2017-10-06 23:19:54.0
10004,70.8,0.0,ADNI2,CN,2014-01-16,2011-07-22,30,30,30,29.80330,ADNI2,18,Female,011_S_4105,4105,11,m30,2.488710,2017-08-29 23:07:16.0


In [10]:
# The percentage for -4 and -1 is really small, so in order to make things easier in the imputation, I just delete these
# observations
ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -4].index)
print("Shape of ADNI2 after dropping observations containing -4")
print(ADNI2_clean.shape)
print("\n")

ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -1 ].index)
print("Shape of ADNI2 after dropping observations containing -1")
print(ADNI2_clean.shape)


Shape of ADNI2 after dropping observations containing -4
(6933, 85)


Shape of ADNI2 after dropping observations containing -1
(6690, 85)


In [11]:
#Check the unique values for each predictor. 
for col in ADNI2_clean.columns:
    print('unique values for', col , len(ADNI2_clean[col].value_counts()))

unique values for RID 1154
unique values for PTID 1154
unique values for VISCODE 23
unique values for SITE 58
unique values for COLPROT 1
unique values for ORIGPROT 3
unique values for EXAMDATE 1345
unique values for DX_bl 5
unique values for AGE 293
unique values for PTGENDER 2
unique values for PTEDUCAT 14
unique values for PTETHCAT 2
unique values for PTRACCAT 6
unique values for PTMARRY 4
unique values for APOE4 3
unique values for CDRSB 28
unique values for ADAS11 62
unique values for ADAS13 78
unique values for MMSE 29
unique values for RAVLT_immediate 76
unique values for RAVLT_learning 18
unique values for RAVLT_forgetting 20
unique values for RAVLT_perc_forgetting 82
unique values for FAQ 31
unique values for MOCA 31
unique values for EcogPtMem 57
unique values for EcogPtLang 68
unique values for EcogPtVisspat 51
unique values for EcogPtPlan 29
unique values for EcogPtOrgan 37
unique values for EcogPtDivatt 19
unique values for EcogPtTotal 474
unique values for EcogSPMem 57
un

Hi guys, <br>
I went through all the predictors. It took me a while to go through all the predictors but here below you see their description. <br>
Here below there is my strategy; I will discuss it with Hsiang and we will implement it <br>
** DOUBLE CLICK on it to see it**




                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 

In [12]:
#Drop columns
print(ADNI2_clean.shape)
ADNI2_clean.drop (['M','Month', 'Month_bl','update_stamp',
                                'FSVERSION', 'EXAMDATE','ORIGPROT' ,
                                'COLPROT', 'SITE','ORIGPROT'], axis =1, inplace= True)

(6690, 85)


In [25]:
ADNI2_clean['EcogSPTotal_bl'].unique()

array([ 1.25641,  1.25   ,  1.37838,  1.05128,  2.83333,  1.17949,
        1.     ,  1.12821,      nan,  1.10256,  1.4375 ,  1.02564,
        2.05882,  2.     ,  3.42857,  1.48718,  1.30769,  1.20513,
        3.4359 ,  1.46154,  1.54054,  1.92308,  1.72222,  1.15385,
        2.33333,  3.55263,  1.51282,  3.76923,  1.19444,  1.61538,
        2.64103,  1.76923,  1.29167,  1.02632,  1.07692,  1.48649,
        2.94444,  1.5    ,  3.34211,  1.82051,  1.83784,  1.71875,
        1.4359 ,  1.64865,  1.33333,  1.69231,  3.05263,  2.89744,
        1.78947,  1.39474,  1.18919,  1.23077,  2.15385,  1.34375,
        1.6    ,  1.2    ,  1.38462,  1.41026,  1.63889,  3.46154,
        1.36842,  2.5641 ,  1.65789,  2.46154,  2.72973,  1.22222,
        1.90323,  1.79487,  1.28571,  2.81579,  2.14286,  1.9    ,
        2.5    ,  2.82051,  2.74359,  2.69231,  1.125  ,  2.31429,
        2.82857,  1.84615,  3.35897,  2.64865,  3.53846,  3.     ,
        1.73529,  2.25641,  1.73684,  1.89189,  2.97436,  2.94

'AGE'
'PTEDUCAT'
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'


In [14]:
#Hot one encoding
#categorical_columns_g2cols = ['DX_bl', 'PTGENDER', 'PTETHCAT','PTRACCAT','PTMARRY','APOE4']                                 
#ADNI2_clean = pd.get_dummies(ADNI2_clean, columns=categorical_columns_g2cols, prefix = categorical_columns_g2cols , drop_first=True)

In [15]:
print(ADNI2_clean.columns)

Index(['RID', 'PTID', 'VISCODE', 'DX_bl', 'AGE', 'PTGENDER', 'PTEDUCAT',
       'PTETHCAT', 'PTRACCAT', 'PTMARRY', 'APOE4', 'CDRSB', 'ADAS11', 'ADAS13',
       'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Hippocampus',
       'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl', 'CDRSB_bl', 'ADAS11_bl',
       'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl', 'RAVLT_learning_bl',
       'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl', 'FAQ_bl',
       'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl', 'Hippocampus_bl',
       'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl', 'MidTemp_bl', 'ICV_bl',
       'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl', 'EcogPtVisspat_bl',
       'EcogPtPlan_bl', 'EcogPtOrgan_bl', '

In [16]:
#Function to split the data in train and test
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test



In [17]:
# Imputation by mean

In [18]:
# Imputation by model
